!["question"](question.jpg)



between layer 1 and 2:

$$
\begin{bmatrix}
  b_{10} & w_{11} & w_{21} \\
  b_{20} & w_{12} & w_{22} 
\end{bmatrix}
$$

where b = bias and w = weights

$x = [0.1,0.2]$

In [44]:
import numpy as np
from random import seed

In [120]:
x = np.array([[0.1,0.2],
      [0.3,0.8]]) # x1, x2

# x = [[0.1,0.2]]

input is columns, output is rows 

In [100]:
def Linear(num_nodes_input_layer, num_nodes_output_layer):
    """Dimensions: output is rows, input is columns."""
    seed(1)
    weights = np.random.rand(num_nodes_output_layer, num_nodes_input_layer + 1)
    # biases = np.ones((weights.shape[0], 1))

    # Stack the original array and the column of ones
    # return np.column_stack((biases, weights ))
    return weights


In [103]:
## define the layers
seed(1)
input = Linear(2,2)
seed(1)
hidden_layer1 = Linear(2,2)
seed(1)
output_layer = Linear(2,2)

hidden_layer1

array([[0.83057883, 0.44581712, 0.35657956],
       [0.09503414, 0.49783754, 0.6120248 ]])

In [104]:
def sigmoid(z):
    return np.array([1/(  1 + np.exp( -linear_combination )  ) for linear_combination in  z])

x = [[1.  0.1 0.2]  <br>
     [1.  0.3 0.8]] <br>

layer = [[b10 w11 w12], <br>
         [b20 w21 w22]] <br>

 
[1.  0.1 0.2] [b10 w11 w12] <br>
[1.  0.3 0.8] [b10 w11 w12] <br>
<br><br>
[1.  0.1 0.2] [b20 w21 w22] <br>
[1.  0.3 0.8] [b20 w21 w22] <br>

$z^{(2)}_1$ = [ <[1.  0.1 0.2], [b10 w11 w12]>  , <[1.  0.3 0.8] [b10 w11 w12]>] <br>
$z^{(2)}_2$ =  [<[1.  0.1 0.2], [b20 w21 w22]>, <[1.  0.3 0.8], [b20 w21 w22]>]

**z** = [$z^{(2)}_1$, $z^{(2)}_2$ ]


In [144]:
def compute_forward(inputs, layer):
    ## linear combination w0x0 + w1x1 + w2x2
    linear_combinations = []

    inputs = np.c_[ np.ones(x.shape[1]), x  ]    ## add bias to the inputs
    print(inputs)
    print(layer)
    ## for each row in x, compute z for every node
    for node in layer:
    #     print(node)
    #     print("----")
        for row, weights_bias in zip(inputs, node):
            print(row, node)
        print("-------------------------")
        linear_combinations.append([ row @ node  for row, weights_bias in zip(inputs, node)]) 


    print(linear_combinations)

    # sigmoids = [1/(  1 + np.exp( -linear_combination )  ) for linear_combination in  linear_combinations] ## compute a
    # print(linear_combinations)
    # print(sigmoids)

    # return sigmoid(linear_combinations)

In [145]:
a2 = compute_forward(x, input)

# a3 = compute_forward(a2, hidden_layer1)

# output = compute_forward(a3, output_layer)
# a2, a3, output

[[1.  0.1 0.2]
 [1.  0.3 0.8]]
[[0.69461823 0.21704253 0.32161864]
 [0.57392364 0.09668728 0.35794559]]
[1.  0.1 0.2] [0.69461823 0.21704253 0.32161864]
[1.  0.3 0.8] [0.69461823 0.21704253 0.32161864]
-------------------------
[1.  0.1 0.2] [0.57392364 0.09668728 0.35794559]
[1.  0.3 0.8] [0.57392364 0.09668728 0.35794559]
-------------------------
[[0.7806462087723156, 1.0170258999350974], [0.6551814877191733, 0.889286298808283]]


In [97]:
def binary_cross_entropy_loss(output, target):
    
    ## m is number of samples
    ## k is total number of output units
    print(output)
    for row in output:
        print(row)
        # for unit in row: ## two output units
        #     print(unit)


In [98]:
binary_cross_entropy_loss(output, x)

[0.62866359 0.79421103]
0.6286635948125413
0.7942110298247739


In [ ]:
def compute_backpropogation()

In [ ]:
def forward(inputs):
    inputs = in

array([[0.13692008, 0.4647963 ],
       [0.65395656, 0.00602726]])

In [76]:
import torch
import torch.nn as nn
import torch.optim as optim

In [87]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()


        self.input = nn.Linear(2,2)
        self.hidden_layer1 = nn.Linear(2,2)
        self.output_layer = nn.Linear(2,2)

    def forward(self, x):
        x = self.input(x)
        x = torch.sigmoid(x)
        x = self.hidden_layer1(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)

        return x
    
model = ANN()

x_vec = torch.tensor( [0.1,0.2] )
target = torch.tensor([1.0, 2])

# ----- Forward Propagation -----
output = model(x_vec)

# ----- Compute Loss -----
criterion = nn.BCEWithLogitsLoss()
loss = criterion(output, target)

# ----- Backpropagation -----
optimizer = optim.SGD(model.parameters(), lr = 0.01)
optimizer.zero_grad() ## remove old gradients
loss.backward()
optimizer.step()



